In [3]:
import basic_tools
from config import DockerConfig
from media import Media
from openai_api import GPTApi, WisperApi
from subtitle import SubtitleWriter, SubtitleDecoder



cfg = DockerConfig()
gpt = GPTApi(cfg("OPENAI_API_KEY_PETER"))
wis = WisperApi(cfg("OPENAI_API_KEY_PETER"))



读取本地配置文件: config.ini


In [14]:
audio_file = open(r"C:\Users\lucyc\Desktop\aaaa.mp3", "rb")
transcript = wis.transcribe_timestamp(audio_file)

In [15]:
print(transcript)

Transcription(text="Face Aerobics to go and have a look at that video. In that video it shows you a number of things. The first thing it shows you and teaches you is how much muscle effort to put into the exercises. So when I do the exercises you can see me just lifting my forehead, you can see me just lifting my cheeks, you can see me just going like this, okay. And as I do those things I'm squeezing the muscles as I do them, okay. I'm not going like this, I'm not doing this, I'm not, I'm not doing this, okay. And I know my face is already contorted and all of that but I do know people who do this. That is not the way to do face aerobics. The way to perform face aerobics is in this video here. All of the effort that you put into the exercises is in squeezing the muscles, not contorting the muscles. So have a think about the size of the muscles, okay. So the forehead muscle is quite a large muscle and it's quite a strong muscle. So you can see how much effort I'm putting into it, okay.

In [41]:
words = transcript.words
print(words)

[{'word': 'Face', 'start': 0.0, 'end': 0.4000000059604645}, {'word': 'Aerobics', 'start': 0.4000000059604645, 'end': 0.8799999952316284}, {'word': 'to', 'start': 0.8799999952316284, 'end': 1.340000033378601}, {'word': 'go', 'start': 1.340000033378601, 'end': 1.5800000429153442}, {'word': 'and', 'start': 1.5800000429153442, 'end': 1.9199999570846558}, {'word': 'have', 'start': 1.9199999570846558, 'end': 1.940000057220459}, {'word': 'a', 'start': 1.940000057220459, 'end': 2.180000066757202}, {'word': 'look', 'start': 2.180000066757202, 'end': 2.2200000286102295}, {'word': 'at', 'start': 2.2200000286102295, 'end': 2.380000114440918}, {'word': 'that', 'start': 2.380000114440918, 'end': 2.5399999618530273}, {'word': 'video', 'start': 2.5399999618530273, 'end': 2.9000000953674316}, {'word': 'In', 'start': 4.340000152587891, 'end': 4.480000019073486}, {'word': 'that', 'start': 4.480000019073486, 'end': 4.679999828338623}, {'word': 'video', 'start': 4.679999828338623, 'end': 4.940000057220459}

In [17]:
sentence = ""
for word in words:
    print(word["word"], end=" ")
    sentence += word["word"] + " "


Face Aerobics to go and have a look at that video In that video it shows you a number of things The first thing it shows you and teaches you is how much muscle effort to put into the exercises So when I do the exercises you can see me just lifting my forehead you can see me just lifting my cheeks you can see me just going like this okay And as I do those things I'm squeezing the muscles as I do them okay I'm not going like this I'm not doing this I'm not I'm not doing this okay And I know my face is already contorted and all of that but I do know people who do this That is not the way to do face aerobics The way to perform face aerobics is in this video here All of the effort that you put into the exercises is in squeezing the muscles not contorting the muscles So have a think about the size of the muscles okay So the forehead muscle is quite a large muscle and it's quite a strong muscle So you can see how much effort I'm putting into it okay So what I'm doing is as I'm lifting I'm squ

In [18]:
prompt = """
You are a subtitle segmenter. Your task is to segment the text enclosed in <<<>>> symbols into short sections based on semantics and pauses, with each section not exceeding 10 words and containing no punctuation. Each segmented subtitle should be presented in the following format:
-
subtitle text 1
-
subtitle text 2
-
Maintain the natural flow of the dialogue and ensure each segment can independently convey a complete meaning. Do not provide any explanations with your outputs. Do not modify any text.

"""

In [19]:
usermsg = "<<<{}>>>".format(sentence)

In [20]:
# openAI message format
message = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": usermsg}
]
message

[{'role': 'system',
  'content': '\nYou are a subtitle segmenter. Your task is to segment the text enclosed in <<<>>> symbols into short sections based on semantics and pauses, with each section not exceeding 10 words and containing no punctuation. Each segmented subtitle should be presented in the following format:\n-\nsubtitle text 1\n-\nsubtitle text 2\n-\nMaintain the natural flow of the dialogue and ensure each segment can independently convey a complete meaning. Do not provide any explanations with your outputs. Do not modify any text.\n\n'},
 {'role': 'user',
  'content': "<<<Face Aerobics to go and have a look at that video In that video it shows you a number of things The first thing it shows you and teaches you is how much muscle effort to put into the exercises So when I do the exercises you can see me just lifting my forehead you can see me just lifting my cheeks you can see me just going like this okay And as I do those things I'm squeezing the muscles as I do them okay I'

In [32]:
response = gpt.query(message, max_tokens=1000, temperature=0.1, model="gpt-4-turbo")

In [68]:
res_sentences = response[2:].split("\n-\n")

In [69]:
res_sentences

['Face Aerobics to go and have a look',
 'at that video In that video it shows you',
 'a number of things The first thing it shows you',
 'and teaches you is how much muscle effort',
 'to put into the exercises So when I do the exercises',
 'you can see me just lifting my forehead',
 'you can see me just lifting my cheeks',
 'you can see me just going like this okay',
 "And as I do those things I'm squeezing the muscles",
 "as I do them okay I'm not going like this",
 "I'm not doing this I'm not I'm not doing this okay",
 'And I know my face is already contorted',
 'and all of that but I do know people who do this',
 'That is not the way to do face aerobics',
 'The way to perform face aerobics is in this video here',
 'All of the effort that you put into the exercises',
 'is in squeezing the muscles not contorting the muscles',
 'So have a think about the size of the muscles okay',
 'So the forehead muscle is quite a large muscle',
 "and it's quite a strong muscle So you can see",
 "ho

In [70]:
nres = " ".join(res_sentences)
if len(nres) != len(sentence):
    print("Lengths do not match")
for i in range(len(nres)):
    if nres[i] == sentence[i]:
        pass
    else:
        print(nres[i], sentence[i])
print("Check complete") 

Lengths do not match
Check complete


In [ ]:
orig_word_list = sentence.split()
orig_index = 0
time_stamps = []
sentences = []

for res_sent in res_sentences:
    time = []
    word_list = res_sent.split()

    if word_list[0] == orig_word_list[orig_index]:
        time.append(words[orig_index]["start"])

    for i in range(len(word_list)):
        if 